In [ ]:
import cv2
import easyocr
import numpy as np
import os, re, time


IMAGE_PATH = "plaka11.jpg"
DEBUG = True          
GPU = False
MIN_CONF = 0.25      
MAX_CANDIDATES = 30   


reader = easyocr.Reader(['tr','en'], gpu=GPU, verbose=False)


def clean_text(t):
    return re.sub(r'[^A-Z0-9]', '', t.upper())

def looks_like_plate(t):
    
    if not (5 <= len(t) <= 10):
        return False
    has_d = any(c.isdigit() for c in t)
    has_a = any(c.isalpha() for c in t)
    return has_d and has_a

def preprocess_variants(roi_gray):
   
    out = []

  
    r1 = cv2.resize(roi_gray, None, fx=3.0, fy=3.0, interpolation=cv2.INTER_CUBIC)
    r1 = cv2.GaussianBlur(r1, (3,3), 0)
    _, t1 = cv2.threshold(r1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    out.append(t1)

   
    r2 = cv2.resize(roi_gray, None, fx=3.0, fy=3.0, interpolation=cv2.INTER_CUBIC)
    r2 = cv2.bilateralFilter(r2, 9, 75, 75)
    t2 = cv2.adaptiveThreshold(r2, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                              cv2.THRESH_BINARY, 31, 8)
    out.append(t2)

    
    r3 = cv2.resize(roi_gray, None, fx=3.0, fy=3.0, interpolation=cv2.INTER_CUBIC)
    _, t3 = cv2.threshold(r3, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    out.append(t3)

    return out

def score_candidate(text, conf):
    """Metne + güvene göre puan."""
    t = clean_text(text)
    if not looks_like_plate(t):
        return -1
   
    ideal = 7
    length_penalty = abs(len(t) - ideal) * 0.06
    return conf - length_penalty


if not os.path.exists(IMAGE_PATH):
    raise FileNotFoundError(f"Dosya yok: {IMAGE_PATH}")

img0 = cv2.imread(IMAGE_PATH)
if img0 is None:
    raise ValueError("Resim okunamadı (format bozuk olabilir).")


target_w = 1000
scale = target_w / img0.shape[1]
img = cv2.resize(img0, (target_w, int(img0.shape[0]*scale)))

h, w = img.shape[:2]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if DEBUG:
    cv2.imshow("0-Orijinal", img)
    cv2.waitKey(1)


rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 5))
sqKern   = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)

gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
gradX = (255 * (gradX - gradX.min()) / (gradX.max() - gradX.min() + 1e-6)).astype("uint8")

gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)

thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKern)
thresh = cv2.erode(thresh, None, iterations=1)
thresh = cv2.dilate(thresh, None, iterations=2)

if DEBUG:
    cv2.imshow("1-Threshold", thresh)
    cv2.waitKey(1)

cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

candidates = []
for c in cnts[:200]:
    x, y, cw, ch = cv2.boundingRect(c)
    area = cw * ch
    aspect = cw / float(ch + 1e-6)

    
    if area < 800:
        continue
    if not (2.0 < aspect < 8.5):
        continue
    if cw < 80 or ch < 20:
        continue

   
    pad = 8
    x1 = max(0, x - pad); y1 = max(0, y - pad)
    x2 = min(w, x + cw + pad); y2 = min(h, y + ch + pad)

    candidates.append((x1, y1, x2, y2, area))


candidates = sorted(candidates, key=lambda t: t[4], reverse=True)[:MAX_CANDIDATES]

if DEBUG:
    dbg = img.copy()
    for (x1,y1,x2,y2,_) in candidates:
        cv2.rectangle(dbg, (x1,y1), (x2,y2), (255,0,0), 2)
    cv2.imshow("2-Adaylar (Mavi)", dbg)
    cv2.waitKey(1)


best = None  # (score, plate, conf, (x1,y1,x2,y2), best_img)
for idx, (x1, y1, x2, y2, _) in enumerate(candidates):
    roi = gray[y1:y2, x1:x2]
    if roi.size == 0:
        continue

    variants = preprocess_variants(roi)

    for v in variants:
        try:
            results = reader.readtext(
                v,
                allowlist="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ",
                paragraph=False,
                detail=1
            )
        except:
            continue

        for (_, text, conf) in results:
            t = clean_text(text)
            sc = score_candidate(t, conf)
            if sc < 0:
                continue

            if (best is None) or (sc > best[0]):
                best = (sc, t, conf, (x1,y1,x2,y2), v)


if best is None:
    print("\n❌ Plaka okunamadı.")
    if DEBUG:
        print("Debug açık: '1-Threshold' ve '2-Adaylar' pencerelerine bak.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
else:
    sc, plate, conf, (x1,y1,x2,y2), best_img = best
    print("\n✅ PLAKA BULUNDU")
    print("Plaka :", plate)
    print(f"Güven : %{conf*100:.1f}")

    out = img.copy()
    cv2.rectangle(out, (x1,y1), (x2,y2), (0,255,0), 3)
    cv2.rectangle(out, (x1, max(0,y1-35)), (x2, y1), (0,255,0), -1)
    cv2.putText(out, plate, (x1+6, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)

    if DEBUG:
        cv2.imshow("3-Sonuc (Yesil)", out)
        cv2.imshow("4-OCR En iyi goruntu", best_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


C:\Users\tb\.conda\envs\env_bst4\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



❌ Plaka okunamadı.
Debug açık: '1-Threshold' ve '2-Adaylar' pencerelerine bak.
